<table width="100%" border="3">
  <tbody>
    <tr>
      <td width=120><img src="https://is4-ssl.mzstatic.com/image/thumb/Purple128/v4/9f/38/cb/9f38cb51-c606-6fd4-54b3-47bfdb06b712/source/256x256bb.jpg" alt="Aidea" width="120"/></td>
      <td align='left'><h1>TensorFlow 2 Object Detection</h1><BR><h2>D1-UseTF2ObjectDetectionModel</h2></td>
    </tr>
  </tbody>
</table>

# Part 1: Set up the Colab environment to execute Object Detection API with TensorFlow 2


## Step 1: Set up Google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

## Step 2: Change working directory to cloud drive

In [ ]:
from os import chdir
chdir("/content/drive/My Drive/")

## Step 3: Use the Git clone command to download the Object Detection API

In [ ]:
!git clone https://github.com/tensorflow/models

In [ ]:
!ls /content/drive/MyDrive/models/research

## Step 4: Clear git settings

In [ ]:
%%bash
cd models
rm -rf .git

## Step 5: Installation packages

In [ ]:
!pip install tf_slim
!pip install lvis
!pip install tensorflow-addons

## Step 6: Set up the working environment

In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.

In [ ]:
%env PYTHONPATH=/env/python:/content/drive/My Drive/models:/content/drive/My Drive/models/research:/content/drive/My Drive/models/research/slim

In [ ]:
import sys
sys.path.append('/content/drive/My Drive/models')

In [ ]:
from os import chdir
chdir("/content/drive/My Drive/models/research/")

## Step 7: Test Object Detection API


In [ ]:
!python object_detection/builders/model_builder_tf2_test.py

# Part 2: Test object detection

## Step 1: Download the object detection model pre-trained by CoCo dataset from the TensorFlow website
https://cocodataset.org/

https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

In [ ]:
%%bash
wget http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d0_coco17_tpu-32.tar.gz
tar -xvf efficientdet_d0_coco17_tpu-32.tar.gz
rm -rf efficientdet_d0_coco17_tpu-32.tar.gz

In [ ]:
!pwd

In [ ]:
!ls

## Step 2: Set the function library required for the execution environment


In [ ]:
import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.builders import model_builder

%matplotlib inline

In [ ]:
def load_image_into_numpy_array(path):
  #Load an image from file into a numpy array.
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  image = image.convert("RGB")
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def get_keypoint_tuples(eval_config):
  """Return a tuple list of keypoint edges from the eval config.
  
  Args:
    eval_config: an eval config containing the keypoint edges
  
  Returns:
    a list of edge tuples, each in the format (start, end)
  """
  tuple_list = []
  kp_list = eval_config.keypoint_edge
  for edge in kp_list:
    tuple_list.append((edge.start, edge.end))
  return tuple_list

## Step 3: Setting up the model

In [ ]:
import os
pipeline_config = 'efficientdet_d0_coco17_tpu-32/pipeline.config'
model_dir = 'efficientdet_d0_coco17_tpu-32/checkpoint'

# Load pipeline config and build a detection model
configs = config_util.get_configs_from_pipeline_file(pipeline_config)

In [ ]:
def get_model_detection_function(model):
  """Get a tf.function for detection."""
  @tf.function
  def detect_fn(image):
    """Detect objects in image."""
    image, shapes = model.preprocess(image)
    prediction_dict = model.predict(image, shapes)
    detections = model.postprocess(prediction_dict, shapes)
    return detections, prediction_dict, tf.reshape(shapes, [-1])

  return detect_fn

In [ ]:
model_config = configs['model']
detection_model = model_builder.build(model_config=model_config, is_training=False)
detect_fn = get_model_detection_function(detection_model)

In [ ]:
# Restore checkpoint
ckpt = tf.compat.v2.train.Checkpoint(model=detection_model)
ckpt.restore(os.path.join(model_dir, 'ckpt-0')).expect_partial()

In [ ]:
label_map_path = configs['eval_input_config'].label_map_path
label_map = label_map_util.load_labelmap(label_map_path)
categories = label_map_util.convert_label_map_to_categories(
    label_map,
    max_num_classes=label_map_util.get_max_label_map_index(label_map),
    use_display_name=True)
category_index = label_map_util.create_category_index(categories)
label_map_dict = label_map_util.get_label_map_dict(label_map, use_display_name=True)

## Step 4: Import image files and convert to Tensor

In [ ]:
image_dir = 'object_detection/test_images/'
image_path = os.path.join(image_dir, 'image3.jpg')
image_np = load_image_into_numpy_array(image_path)

In [ ]:
test_image = Image.open(image_path)
test_image=test_image.resize((512, 512))
display(test_image)

In [ ]:
image_np.shape

In [ ]:
input_tensor = tf.convert_to_tensor(
    np.expand_dims(image_np, 0), dtype=tf.float32)

In [ ]:
input_tensor.shape

In [ ]:
type(input_tensor)

## Step 5: Do detection

In [ ]:
detections, predictions_dict, shapes = detect_fn(input_tensor)

In [ ]:
print(detections["detection_scores"])

## Step 6: Check detection result

In [ ]:
label_id_offset = 1
image_np_with_detections = image_np.copy()

viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_detections,
      detections['detection_boxes'][0].numpy(),
      (detections['detection_classes'][0].numpy() + label_id_offset).astype(int),
      detections['detection_scores'][0].numpy(),
      category_index,
      use_normalized_coordinates=True,
      max_boxes_to_draw=200,
      min_score_thresh=.30,
      agnostic_mode=False)

plt.figure(figsize=(12,16))
plt.imshow(image_np_with_detections)
plt.show()